In [1]:
# Libary to import self-made libaries
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Libaries')

In [2]:
# Importing self-made libaries
import standards as s
import life_cycle_assessment as lc
import LCA_plots as lp
import reload_lib as rl
import non_bio_co2 as nbc


lib = [s, lc, lp, nbc]

In [3]:
from copy import deepcopy as dc

In [19]:
import bw2io as bi
import bw2data as bd
import pandas as pd

In [5]:
bd.projects.set_current("single use vs multi use")


In [35]:
if any("biosphere" in db for db in bd.databases):
    print('Biosphere is already present in the project.')
else:
    bi.bw2setup()

Biosphere is already present in the project.


In [36]:
if 'ev391cutoff' in bd.databases and 'ev391consq' in bd.databases and 'ev391apos' in bd.databases:
    print('Ecoinvent 3.9.1 is already present in the project.')
else:
    # APOS
    ei = bi.SingleOutputEcospold2Importer(dirpath=r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_apos_ecoSpold02\datasets", db_name='ev391apos') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

    # Consequential
    ei = bi.SingleOutputEcospold2Importer(dirpath=r"C:\Users\ruw\Desktop\4. semester\EcoInvent\ecoinvent 3.9.1_consequential_ecoSpold02\datasets", db_name='ev391consq') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

    # cut-off
    ei = bi.SingleOutputEcospold2Importer(dirpath=r"C:\Users\ruw\Downloads\ecoinvent 3.9.1_cutoff_ecoSpold02\datasets", db_name='ev391cutoff') #recommendation for consistent databases naming: database name (ecoinvent), version number, system model
    ei.apply_strategies() #fixing some issues when ecoinvent and brightway have to talk together by going through all datasets and manipulating them in a specific way
    ei.statistics() #checking if everything worked out with strategies and linking
    ei.write_database() #save the database to our hard drive

Ecoinvent 3.9.1 is already present in the project.


In [55]:
db_path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\use_phase.xlsx'
sheet_name = ['ev391apos', 'ev391consq', 'ev391cutoff'] # Replace with your actual sheet name


In [81]:
db_path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\EoL_case1.xlsx' 
sheet_name = ['ev391apos', 'ev391consq', 'ev391cutoff'] # Replace with your actual sheet name
for db in sheet_name:
    # Read the Excel file
# Read the specific sheet from the Excel file
    data = pd.read_excel(db_path, sheet_name=db)
    # Save the data to a temporary file that can be used by ExcelImporter
    temp_path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Data\databases\temp_use_phase.xlsx'
    data.to_excel(temp_path, index=False)



    # Use the temporary file with ExcelImporter
    imp = bi.ExcelImporter(temp_path)  # the path to your inventory excel file
    imp.apply_strategies()

    # # List of databases to match

    # Loop through each database and match
    print(f"Matching database: {db}")
    imp.match_database(db, fields=('name', 'unit', 'location', 'reference product'))
    print(f"Unlinked items after matching {db}: {list(imp.unlinked)}")

    # Match without specifying a database
    imp.match_database(fields=('name', 'unit', 'location'))

    # Generate statistics and write results
    imp.statistics()
    imp.write_excel(only_unlinked=True)
    unlinked_items = list(imp.unlinked)
    imp.write_database()

    # Print unlinked items if needed
    print(unlinked_items)

Extracted 1 worksheets in 0.04 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 4.93 seconds
Matching database: ev391apos
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


Unlinked items after matching ev391apos: []
Applying strategy: link_iterable_by_fields
10 datasets
77 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\ruw\AppData\Local\pylca\Brightway3\single-use-vs-multi-use.0e7e263503916b8fcd098a9e51faa14f\output\db-matching-eol_case1_apos-unlinked.xlsx


0% [##########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/20/2024 15:27:55
  Finished: 12/20/2024 15:27:55
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 3.01
Created database: eol_case1_apos
[]
Extracted 1 worksheets in 0.03 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
App

TypeError: 'int' object is not iterable

Matching database: ev391apos
Applying strategy: link_iterable_by_fields
Unlinked items after matching ev391apos: []
Matching database: ev391consq
Applying strategy: link_iterable_by_fields
Unlinked items after matching ev391consq: []
Matching database: ev391cutoff
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


Unlinked items after matching ev391cutoff: []
Applying strategy: link_iterable_by_fields
9 datasets
51 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\ruw\AppData\Local\pylca\Brightway3\single-use-vs-multi-use.0e7e263503916b8fcd098a9e51faa14f\output\db-matching-use_phase-unlinked.xlsx


0% [#########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 12/20/2024 08:28:09
  Finished: 12/20/2024 08:28:09
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 2.04
Created database: use_phase
[]
